# Information Retrieval

This week, we will learn some basics of information retrieval, and build a simple search engine.

## Hamlet sentences

We want to build a full text search index for Hamlet in this assignment.

First load the Hamlet data from the previous assignment, and split it into sentences. Beware of the particular structure of this document, which not only separates sentences with a dot.

Then tokenize the sentences as in the previous assignment, such that each sentence is a sequence of *words* (no punctuation tokens). Do *not* remove stopwords.

In [1]:
import gzip, re
import string

_sentence_pattern = re.compile("[.:!;\n]\s", re.U)                                                    
_words = re.compile(r"[\w']+", re.U)

# Read the entire file:
with open("../data/hamlet.txt") as file: 
    full = file.read()

sentences = list() # Store your output in this list

# First split Hamlet into sentences, then tokenize each sentence.
for line in re.split(_sentence_pattern, full):
    print(line)
    break
    if line.strip() != '':
        line = line.translate(str.maketrans('', '', string.punctuation)).strip()
        sentences.append(line.lower().split())

print("Hamlet contains %d sentences, %d tokens." % (len(sentences), sum(len(s) for s in sentences)))

THE TRAGEDY OF HAMLET, PRINCE OF DENMARK
Hamlet contains 0 sentences, 0 tokens.


Find the longest sentence (as an array of tokens) and print it

In [18]:
longest = [] # store the answer here, as array

longest = max(sentences, key=len)

# raise NotImplementedError()
print("Length of longest sentence:", len(longest))
print(*longest)

Length of longest sentence: 19
enter claudius king of denmark gertrude the queen hamlet polonius laertes and his sister ophelia voltemand cornelius lords attendant


In [19]:
### AUTOMATIC TESTS ###
assert len(longest) > 18, "There should be a longer sentence."
assert len(longest) < 180, "A sentence appears a bit overly long."

Count how many sentences have exactly one token. Why are there so many? Find the 10 most frequent one-word sentences.

In [21]:
singletons = 0 # Store your answer in this variable

for s in sentences:
    if len(s) == 1:
        singletons += 1

print("There are %d sentences with just one word.\n" % singletons)

most_common = [] # Store the 10 most common one-word sentences and their counts

from collections import Counter

# Print the most common one-word sentences:
one_word_sentences = [word for s in sentences for word in s if len(s) == 1]

Counter = Counter([word for s in sentences for word in s if len(s) == 1])
most_common = Counter.most_common(10)

for word, count in most_common:
    print(word, count, sep="\t")

There are 1375 sentences with just one word.

ham	358
hor	108
king	106
pol	86
queen	74
laer	62
oph	58
ros	45
clown	33
mar	31


In [22]:
### AUTOMATIC TESTS ###
assert singletons > 1000, "There should be more one-word sentences."
assert singletons < 1500, "There should be fewer one-word sentences."
assert len(most_common) == 10, "You are supposed to return 10 only."
assert sum([c for _,c in most_common]) > 800, "The most common should cover more."
assert sum([c for _,c in most_common]) < 1000, "The most common should cover less."

## Build an inverted index

For full-text search, we need an inverted index. Build a lookup table that allows us to find all sentence numbers that contain a particular word. Do not include multiple occurrences.

In [24]:
from collections import defaultdict, OrderedDict
from collections import Counter

index = defaultdict(list) # words to occurrences


for i, s in enumerate(sentences):
    for w in set(s):
        index[w].append(i)

for k, v in index.items():
    index[k] = sorted(list(set(v)))
    
print("The index contains %d words and %d occurrences" % (len(index), sum([len(x) for x in index.values()])))

The index contains 4790 words and 31115 occurrences


In [9]:
### AUTOMATIC TESTS ###
assert "the" in index, "You probably removed stopwords"
assert not "man," in index, "There is still punctuation included"
assert len(index) == len(set([x for s in sentences for x in s])), "You have lost some words."
assert len(index) > 4000, "You have very few words"
assert len(index) < 5500, "You have very many words"
for x in index.values(): assert x == sorted(x), "Lists are not sorted."
for x in index.values(): assert len(x)==len(set(x)), "You have duplicates."
assert sum([len(x) for x in index.values()]) < sum([len(s) for s in sentences]), "You have duplicates."
assert max([len(x) for x in index.values()]) > 500, "Index seems to miss a frequent word."
assert max([len(x) for x in index.values()]) < 1000, "Duplicate indexing?"

# Excursus: Generators in Python

Python has a (rather uncommon) powerful feature called [*generators*](https://wiki.python.org/moin/Generators).

- When writing generators, they are like functions that can "return" multiple values (using `yield`), and will be paused inbetween
- When consuming generators, they behave essentially like an iterator
- Generators are *lazy*: they do *not* produce a list of all their output, but always one item when necessary
- Generators *could* produce an infinite stream of values

In the following assignments, please use generators for efficiency. Here is a simple example how generators work:

In [10]:
def upto(x):
    i = 0
    while i <= x:
        print("gen: generating", i)
        yield i # Return value and pause!
        print("gen: continuing")
        i += 1

print("Use generator in for loop:")
for j in upto(2):
    print("use: generated:", j)
    print("use: next")

print("Use generator object directly:")
a = upto(1)
print("Type of a:", type(a))
print(next(a))
print("Wait")
print(next(a))
try:
    print(next(a))
except StopIteration:
    print("No further values.")
    
print(*upto(2)) # The star expands an iterable/generator

Use generator in for loop:
gen: generating 0
use: generated: 0
use: next
gen: continuing
gen: generating 1
use: generated: 1
use: next
gen: continuing
gen: generating 2
use: generated: 2
use: next
gen: continuing
Use generator object directly:
Type of a: <class 'generator'>
gen: generating 0
0
Wait
gen: continuing
gen: generating 1
1
gen: continuing
No further values.
gen: generating 0
gen: continuing
gen: generating 1
gen: continuing
gen: generating 2
gen: continuing
0 1 2


Write yourself a simple generator to enumerate an existing list: given an input list `[a,b,c]` generate an output containing pairs of `(i,v)` where `i` is the 0-based index of the list.

In [11]:
def my_enumerate(existing):
    """Enumerate the values in the existing list."""
    # YOUR CODE HERE
    count = 0
    for item in existing:
        yield (count, item)
        count += 1

#     raise NotImplementedError()

for item in my_enumerate('test'):
    print(item)

print('Python Enumerate function: ', list(enumerate('test'))) 
print('My Function: ', list(my_enumerate('test')))
    
# for i, string in my_enumerate(["apple", "banana", "coconut"]):
#     print("Index", i, "value", string)

(0, 't')
(1, 'e')
(2, 's')
(3, 't')
Python Enumerate function:  [(0, 't'), (1, 'e'), (2, 's'), (3, 't')]
My Function:  [(0, 't'), (1, 'e'), (2, 's'), (3, 't')]


In [12]:
### AUTOMATIC TESTS ###
assert my_enumerate != enumerate, "You have to implement it yourself!"
assert list(my_enumerate("test")) == list(enumerate("test")), "Does not produce the expected output."
from unittest.mock import patch
with patch('__main__.enumerate') as mock_enumerate:
    list(my_enumerate("test"))
mock_enumerate.assert_not_called()
import types 
assert isinstance(my_enumerate("test"), types.GeneratorType), "Not a generator"

# Intersection of sorted lists

Back to Hamlet: write a *generator* for the *sorted* intersection of two sorted iterators (e.g., list iterators or other generators). Use a **merge** operation as discussed in class!

You may assume that the input is ordered and does not contain duplicates.

In [13]:
def intersect(itera, iterb):
    """Generate the intersection of the two iterators. Do *not* use a list or set!"""
    itera, iterb = iter(itera), iter(iterb)
    try:
        a, b = next(itera), next(iterb)
        while True:
            if a == b:
                yield a
            if a < b:
                a = next(itera)
            else:
                b = next(iterb)
        
    except StopIteration:
        pass # Figure out why this is the right thing to do here!


    
print(*intersect(range(27,51), [7,23,42,99]))
print(*intersect("abc","abc"))
# We want to compute the intersection of intersections!
print(*intersect("abcdef", intersect("cdefgh", "efghij")))

42
a b c
e f


In [14]:
### AUTOMATIC TESTS ###
import types
assert isinstance(intersect("abc","abc"), types.GeneratorType), "Not a generator"
assert sorted(intersect(range(0,10), range(0,10))) == list(range(0,10)), "Result is no longer sorted"
assert len(list(intersect(range(0,10),range(0,10)))) == 10, "Cannot self-intersect"
assert len(list(intersect(range(0,10),range(10,20)))) == 0, "Does not work on disjoint sets"
assert list(intersect("abcdef", intersect("cdefgh", "efghij"))) == ["e","f"], "Does not combine"
from unittest.mock import patch
with patch('__main__.set') as mock_set, patch('__main__.list') as mock_list:
    for v in intersect("abc","abc"): pass
mock_set.assert_not_called()
mock_list.assert_not_called()

## Search!

We want to use above index and functions to find all sentences that contain `hamlet` and `horatio`.

Write a function `search` that, given a list of keywords, finds all sentence containing all of them.

In [15]:

## Correction

def search(*words):
    from functools import reduce
    return reduce(intersect, map(index, get, words))

############################################

def search(*words):
    """Find all sentence numbers that contain each word in `words`"""
    # YOUR CODE HERE
    next_element = index[words[0]]
    for word in words:
        print(word)
        res = intersect(index[word], next_element)
        next_element = res
    
    return res
        
#     raise NotImplementedError()

# print(index['horatio'])
# print()
# print(index['hamlet'])
# print()
# for i, s in enumerate(index['horatio']): print(i, s)



for i,s in enumerate(search("hamlet", "horatio")): print(i,s," ",*sentences[s])
print()
for i,s in enumerate(search("to", "be", "or", "not")): print(i,s," ",*sentences[s])

hamlet
horatio
0 576   enter hamlet horatio marcellus
1 2073   manet hamlet horatio
2 3111   enter hamlet and horatio a farre off
3 3382   enter hamlet and horatio

to
be
or
not
0 1126   but what might you think when i had seene this hot loue on the wing as i perceiued it i must tell you that before my daughter told me what might you or my deere maiestie your queene heere think if i had playd the deske or tablebooke or giuen my heart a winking mute and dumbe or lookd vpon this loue with idle sight what might you thinke no i went round to worke and my yong mistris thus i did bespeake lord hamlet is a prince out of thy starre this must not be
1 1657   to be or not to be that is the question
2 2267   what if this cursed hand were thicker then it selfe with brothers blood is there not raine enough in the sweet heauens to wash it white as snow whereto serues mercy but to confront the visage of offence and whats in prayer but this twofold force to be forestalled ere we come to fall or pardon

In [16]:
### AUTOMATIC TESTS ###
assert list(search("hamlet"))==list(index["hamlet"]), "Not consistent with index"
assert len(list(search("hamlet")))==len(list(search("hamlet","hamlet"))), "hamlet and hamlet"
assert len(list(search("hamlet","horatio")))==4, "Some results are missing"
assert len(list(search("to", "be", "or", "not")))>0, "Some results are missing"
assert len(list(search("to", "be", "or", "not")))<=5, "Unexpectedly many results"
assert list(search(*["ham" for x in range(100)])) == list(index["ham"]), "Does not work with many"
import types
assert isinstance(search("hamlet","hamlet","hamlet"), types.GeneratorType), "Not a generator"
with patch('__main__.intersect') as mock_intersect:
    search("hamlet", "horatio")
mock_intersect.assert_called()

hamlet
hamlet
hamlet
hamlet
hamlet
horatio
to
be
or
not
to
be
or
not
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
ham
hamlet
hamlet
hamlet
hamlet
horatio


## Compute the union

In order to perform "OR" searches, e.g., to find all sentences that contain "hamlet" or "horatio", we need a different merge operation. Also implement the `union` merge using generators as above.

You may assume that the input is ordered and does not contain duplicates.

In [17]:
def union(itera, iterb):
    """Generate the union of the two iterators. Do *not* use a list or set!"""
    def safe_next(i):
        """Helper function because exceptions are not too elegant."""
        try:
            return next(i)
        except StopIteration:
            return None
    
    itera, iterb = iter(itera), iter(iterb)
    a, b = safe_next(itera), safe_next(iterb)
    # YOUR CODE HERE
    while (a is not None) and (b is not None):
        try:
            if a == b:
                yield a
                a = safe_next(itera)
                b = safe_next(iterb)
            elif a < b:
                yield a
                a = safe_next(itera)
            else:
                yield b
                b = safe_next(iterb)
        except TypeError:
            pass
        
    while a is not None:
        yield a
        a = safe_next(itera)
        
    while b is not None:
        yield b
        b = safe_next(iterb) 
    
     
    #raise NotImplementedError()

print(*union([2,4,6],[2,4,6]))
print(*union("abc","abc"))
print(*union(range(0,7), range(4,10)))

2 4 6
a b c
0 1 2 3 4 5 6 7 8 9


In [18]:
### AUTOMATIC TESTS ###
import types
assert isinstance(union("abc","abc"), types.GeneratorType), "Not a generator"
assert sorted(union(range(0,10), range(0,10))) == list(range(0,10)), "Result is no longer sorted"
assert len(list(union(range(0,10),range(0,10)))) == 10, "Cannot self-intersect"
assert len(list(union(range(0,10),range(10,20)))) == 20, "Does not work on disjoint lists"
assert list(union("abcdef", union("cdefgh", "efghij"))) == list("abcdefghij"), "Does not combine"
from unittest.mock import patch
with patch('__main__.set') as mock_set, patch('__main__.list') as mock_list:
    for v in union("abc","abc"): pass
mock_set.assert_not_called()
mock_list.assert_not_called()

## Search with AND and OR

Perform a more complex search using above functions.

Search for all sentences that contain ("hamlet" or "horatio") and "shall"

In [71]:
answer = [] # Store your result in this variable

# First Union:
first_step = union(index['hamlet'], index['horatio'])
answer = intersect(first_step, index['shall'])

# YOUR CODE HERE
# def complex_search(*words):
#     next_word = index[words[0]]
#     for word in words:
#         res = union(index[word], next_word)
#         print('Union of word {}: {}'.format(word, *res))
#         next_word = res
#         if word == words[-1]:
#             print('This is res before intersection: {}\n'.format(*res))
#             res = intersect(index[word], next_word)
    
# #     answer = list(res)
#     return list(res)

# raise NotImplementedError()
answer = list(answer) # in case your answer was a generator
for i,s in enumerate(answer): print(i, s, " ", *sentences[s])

0 869   and what so poore a man as hamlet is may doe t expresse his loue and friending to you god willing shall not lacke
1 2991   but good laertes will you doe this keepe close within your chamber hamlet returnd shall know you are come home
2 3724   oh good horatio what a wounded name things standing thus vnknowne shall liue behind me


In [72]:
# This cell contains a server-side only test to grade the point
assert len(answer)>0, "No results"